# Урок 4. Сверточные нейронные сети
# Домашнее задание

### <span class="burk">Задание 1.</span>
**Сделайте краткий обзор любой архитектуры для object detection.**

Проведите анализ: 
 - Чем отличается выбранная вами архитектура нейронной сети от других?
 - В чём плюсы и минусы данной архитектуры?
 - Какие могут возникнуть трудности при применении этой архитектуры на практике?


### <span class="girk">Выводы:</span>  
 1. Простая смена функции активации с ```relu``` на ```tanh``` + снижение ```Dropout``` в 2 раза позволила увеличить ```accuracy``` на тестовой выборке с 0,5 до 0,55
 2. Уменьшение размера партии c 128 до 64 (batch) уменьшает точность
 3. Замена функции активации в последнем слое активации с ```softmax``` на иную будет снижать точность. 
 4. Помимо конфигурирования слоев необходимо тюнить Dropout под каждую конфигурацию: уменьшение Convolution в первом слое с (3, 3) на (2, 2) и (1, 1) вместе с увеличением Dropout-ов с 0,06 до 0,1 привело к росту точности модели до 0,62.

### <span class="burk">Задание 2$^*$.</span>
**Ссылка на репозиторий с полным кодом для обучения ssd нейросети:
https://github.com/sergeyveneckiy/ssd-tensorflow.** 

**Попробуйте улучшить точность её
работы и напишите отчёт (что вы пробовали изменить в её параметрах и как это
отражалось на процессе обучения нейронной сети)**

**Для MNIST:**  
 - Необходимо понизить количество входящих каналов до 2 (черный и белый)
 - Увеличить количество слоёв
 - Увеличить количество эпох
 - Тюнинг Dropout-ов  

**Для CIFAR100:**  
 - Увеличить количество выходных классов до 100
 - Возможно понадобится пересмотр архитектуры, так как в CIFAR100 классы объединены в суперклассы по 5. Таким образом сеть должна сначала отнести изображение к одному из 20 суперклассов, затем уже определять к какому из классов внутри суперкласса относится тестовое изображение.
 - Свертки и ядра возможно останутся такими же как в текущей модели, однако понадобится увеличение количества слоёв/эпох.
 - Тюнинг Dropout-ов

**Для IMAGENET:**  
 - Для анализа сета скорее всего потребуется ансамбль нейросетей, т.к. датасет очень большой.
 - Следует использовать подход сеть-внутри-сети
 - Учитывая что классов большое количество следует ограничить выборку.
 - Количество выходных классов нужно увеличить, также увеличить свертки/пулинги
 - Также понадобится увеличение количества слоёв/эпох